In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -q langchain
!pip install -q langchain-community
!pip install -q ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00


# Load LLM

In [4]:
!curl https://ollama.ai/install.sh | sh


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  38995      0 --:--:-- --:--:-- --:--:-- 38947
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [5]:
!ollama serve > rocama.log 2>&1 &
!ollama pull aya-expanse

# Read stock tweets data

In [17]:
import json

path = '/content/drive/MyDrive/foolad.json'
with open(path, 'r') as f:
    data = json.load(f)

In [ ]:
data

[{'id': '16683616',
  'body': 'من منتظرم سهم فولاد بشه 1 ریال تا خرید کنم البته فقط 20 هزار تومن',
  'creation_date_time': '2025-06-29T23:55:46+03:30',
  'entity_id': '453',
  'entity_type': 'exchange.asset',
  'entity_info': {'id': '453',
   'type': 'exchange.asset',
   'info': {'trade_symbol': 'فولاد',
    'english_trade_symbol': 'FOLD1',
    'title': 'فولاد مبارکه اصفهان',
    'english_title': 'Mobarakeh Esfehan Steel',
    'short_title': 'فولاد مبارکه اصفهان',
    'english_short_name': 'Mobarakeh Esfehan Steel',
    'slug': 'فولاد_مبارکه_اصفهان'}},
  'account': {'id': '1672538',
   'username': 'Harezai',
   'avatar_id': '',
   'full_name': ''},
  'replies_count': 0,
  'upvote_count': 4,
  'downvote_count': 1,
  'state_id': '1',
  'state': 'updated'},
 {'id': '16683465',
  'body': 'نخرید کهبدبخت میشید \nحتی اگه مجانیم دادن سهم فولاد بازم ضرر میکنید این سهم ورشکستست دوماه هر روز منفی ۳نخرید قحطی سهمه مگه فولاد نخرید',
  'creation_date_time': '2025-06-29T23:23:55+03:30',
  'entity_id'

In [ ]:
len(data)

8000

In [ ]:
data[0]

{'id': '16683616',
 'body': 'من منتظرم سهم فولاد بشه 1 ریال تا خرید کنم البته فقط 20 هزار تومن',
 'creation_date_time': '2025-06-29T23:55:46+03:30',
 'entity_id': '453',
 'entity_type': 'exchange.asset',
 'entity_info': {'id': '453',
  'type': 'exchange.asset',
  'info': {'trade_symbol': 'فولاد',
   'english_trade_symbol': 'FOLD1',
   'title': 'فولاد مبارکه اصفهان',
   'english_title': 'Mobarakeh Esfehan Steel',
   'short_title': 'فولاد مبارکه اصفهان',
   'english_short_name': 'Mobarakeh Esfehan Steel',
   'slug': 'فولاد_مبارکه_اصفهان'}},
 'account': {'id': '1672538',
  'username': 'Harezai',
  'avatar_id': '',
  'full_name': ''},
 'replies_count': 0,
 'upvote_count': 4,
 'downvote_count': 1,
 'state_id': '1',
 'state': 'updated',
 'sentiment': 'مثبت'}

# Prompt template

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

tagging_prompt = ChatPromptTemplate.from_template(
  """به عنوان یک تحلیلگر خبره، احساسات موجود در توییت‌های فارسی درباره بازار بورس ایران را طبقه‌بندی کن. برای توصیف احساسات موجود
  در توییت، تنها از یک کلمه استفاده کن و از ارائه توضیحات اضافی خودداری کن.
کلاس‌ها: مثبت، منفی، خنثی

مثال‌ها:
۱. توییت: "سهام فولاد امروز عملکرد عالی داشت و رشد خوبی کرد"
احساس: مثبت

۲. توییت: "نگران آینده سهام خودرو هستم، به نظر می‌رسد افت ادامه دار خواهد بود"
احساس: منفی

۳. توییت: "شرکت مخابرات امروز گزارش مالی خود را منتشر کرد"
احساس: خنثی

توییت جدید: "{tweet}"
احساس:"""
)



local_model = 'aya-expanse'
llm = ChatOllama(model = local_model)

In [16]:
inp = "پیش بینی رشد خوبی داریم"
prompt = tagging_prompt.invoke({"tweet": inp})
answer = llm.invoke(prompt)
label = answer.content

# Labeling tweets based on the sentiment

In [18]:
from tqdm import tqdm

for each in tqdm(data):
  inp = each['body']
  prompt = tagging_prompt.invoke({"tweet": inp})
  answer = llm.invoke(prompt)
  each['sentiment_label'] = answer.content


with open('/content/drive/MyDrive/foolad_sentiment_labeled.json', 'w', encoding='utf-8-sig') as f:
  json.dump(data, f, indent=2, ensure_ascii=False)